In [5]:
# import sys
# sys.path.append('/Applications/anaconda3/lib/python3.8/site-packages')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import time
import stk
import stko
import spindry as spd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdDistGeom
from rdkit.Chem import rdMolAlign
from rdkit import RDLogger
import logging

# RDkit logger
rdkit_logger = RDLogger.logger()
rdkit_logger.setLevel(RDLogger.CRITICAL)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
%matplotlib inline

def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()



def rdkit_op(bb):
    rdkit_bb = bb.to_rdkit_mol()
    rdkit.SanitizeMol(rdkit_bb)
    rdkit.MMFFOptimizeMolecule(rdkit_bb)

    # stk molecules are immutable. with_position_matrix returns a
    # a clone, holding the new position matrix.
    bb = bb.with_position_matrix(
        position_matrix=rdkit_bb.GetConformer().GetPositions(),
    )

    return bb

In [6]:
def final_porphyrin(bb1,bb2,linker,unit):
    bb1_free = open ('Porphyrin_FG_free.txt','r')
    bb1_free = bb1_free.readlines()
    bb1_smile =[]
    for i in bb1_free:
        bb1_smile = bb1_smile+ [i.replace('\t\n','')]
        for f in bb1_smile:
            bb1 = stk.BuildingBlock(bb1_smile, [stk.BromoFactory()])

    linker = open ('linker_smiles.txt','r')
    linker_smile = linker.readlines()
    linker_smile_edit=[]
    for i in linker_smile:
        linker_smile_edit = linker_smile_edit+ [i.replace('\n','')]
    

  

## FG & Linker & No. units

In [22]:
bb1_free = open ('FG_test.txt','r')
bb1_list = bb1_free.readlines()
bb1_smile =[]
bb3_smile = []
linker = open ('linker_test.txt','r')
linker_smile = linker.readlines()
linker_smile_edit=[]

for i in bb1_list:
    bb1_smile = bb1_smile+ [i.replace('\n','')]

for f in bb1_smile:
    bb1 = stk.BuildingBlock(f, [stk.BromoFactory()])
    # Optimize with the MMFF force field.
    bb1 = rdkit_op(bb1)
    bb3_smile = bb3_smile+ [f.replace('Br/','')]

    for mol2 in bb3_smile:
        bb3 = stk.BuildingBlock(mol2, [stk.BromoFactory()])
        bb3 = rdkit_op(bb3)   

    for a in linker_smile:
        linker_smile_edit = linker_smile_edit+ [a.replace('\n','')]
        for b in linker_smile_edit: 
            linker = stk.BuildingBlock(b,
                functional_groups=[stk.BromoFactory()]
                )
        
        for no in range(0,2):
            repeat_units = no
            repeating_unit_str='CB'+'AB'*repeat_units +'C'
            orientation_str = '0, '*(len(repeating_unit_str)-1)+'1'


            porphyrin_noM = stk.ConstructedMolecule(
                topology_graph=stk.polymer.Linear(
                    building_blocks=(bb1, linker, bb3),
                    repeating_unit=repeating_unit_str,
                    num_repeating_units=1,
                    orientations=tuple(map(int, orientation_str.split(', '))),
                    optimizer=stk.MCHammer(),
                    #optimizer=stk.Collapser(scale_steps=False),
                ),
            )

            show_stk_mol(porphyrin_noM)

        

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# FG & Linker & No.unit & Metal

In [9]:
linker

BuildingBlock('BrC#CC#CBr', (Bromo(Br(2), C(1), bonders=(C(1),), deleters=(Br(2),)), Bromo(Br(5), C(4), bonders=(C(4),), deleters=(Br(5),))))

In [27]:
# Metal centre porphyrin
metal_centers = [
    stk.BuildingBlock(
        smiles='[Fe+2]',
        functional_groups=(
            stk.SingleAtom(stk.Fe(0, charge=2))
            for i in range(6)
        ),
        position_matrix=[[0, 0, 0]],
    ),
    stk.BuildingBlock(
        smiles='[Zn+2]',
        functional_groups=(
            stk.SingleAtom(stk.Zn(0, charge=2))
            for i in range(6)
        ),
        position_matrix=[[0, 0, 0]],
    ),
]

def porphyrin_unit_M(M_smile, metal_atom):
    porphyrin_Br = stk.BuildingBlock(
        smiles= M_smile,
        functional_groups=(
            stk.SmartsFunctionalGroupFactory(
                smarts='[#6!H]~[#7]~[#6!H]',
                bonders=(1,),
                deleters=(),
            ),
            stk.BromoFactory(),
        ),
    )
    porphyrin_Br = stko.UFF().optimize(porphyrin_Br)

    complex_Zn_Br = stk.ConstructedMolecule(
        topology_graph=stk.metal_complex.Porphyrin(
            metals=metal_atom,
            ligands=porphyrin_Br,
            optimizer=stk.MCHammer(),
        )
    )

    bb_Zn = stk.BuildingBlock.init_from_molecule(complex_Zn_Br,
        functional_groups=[stk.BromoFactory(),
        ]
    )

    return bb_Zn


bb1_M = open ('FG_Metal_test.txt','r')
bb1_M_list = bb1_M.readlines()
bb1_M_smile =[]
bb3_M_smile = []
linker = open ('linker_test.txt','r')
linker_smile = linker.readlines()
linker_smile_edit=[]

for i in bb1_M_list:
    bb1_M_smile = bb1_M_smile+ [i.replace('\n','')]


# Over metal atoms
for ma in metal_centers:
    # Initial porphyrin unit with 2 Br
    for f in bb1_M_smile:
        bb1_Zn = porphyrin_unit_M(f, metal_atom=ma)
        # Optimize with the MMFF force field.
        # bb1_Zn = stko.UFF().optimize(bb1_Zn)

        bb3_smile = bb3_smile+ [f.replace('Br/','')]


        for mol in bb3_smile:
            bb3_Zn = porphyrin_unit_M(mol, metal_atom=ma)
            # bb3_Zn = stko.UFF().optimize(bb1_Zn)


        for a in linker_smile:
            linker_smile_edit = linker_smile_edit+ [a.replace('\n','')]
            for b in linker_smile_edit: 
                linker = stk.BuildingBlock(b,
                    functional_groups=[stk.BromoFactory()]
                    )

            for no in range(0,2):
                repeat_units = no
                repeating_unit_str='CB'+'AB'*repeat_units +'C'
                orientation_str = '0, '*(len(repeating_unit_str)-1)+'1'


                porphyrin_Zn = stk.ConstructedMolecule(
                    topology_graph=stk.polymer.Linear(
                        building_blocks=(bb1_Zn, linker, bb3_Zn),
                        repeating_unit=repeating_unit_str,
                        num_repeating_units=1,
                        orientations=tuple(map(int, orientation_str.split(', '))),
                        optimizer=stk.MCHammer(),
                        #optimizer=stk.Collapser(scale_steps=False),
                    ),
                )

                show_stk_mol(porphyrin_Zn)

    


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

...

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
building_block = stk.BuildingBlock('NCCN', (stk.PrimaryAminoFactory(), ))

fgs = building_block.get_functional_groups()
print(building_block)
new_fgs = list(fgs)[1]
building_block = building_block.with_functional_groups((new_fgs, ))
print(building_block)

BuildingBlock('NCCN', (PrimaryAmino(N(0), H(4), H(5), C(1), bonders=(N(0),), deleters=(H(4), H(5))), PrimaryAmino(N(3), H(10), H(11), C(2), bonders=(N(3),), deleters=(H(10), H(11)))))
BuildingBlock('NCCN', (PrimaryAmino(N(3), H(10), H(11), C(2), bonders=(N(3),), deleters=(H(10), H(11))),))


In [30]:
for unit in range (2,10):
        num_repeating_units= unit
print(unit)

9


In [32]:
list(range(2,10))

[2, 3, 4, 5, 6, 7, 8, 9]

In [89]:
# Generate conformers
#def confgen(polymer, output, prunermsthresh, numconf, add_ref):
def confgen(polymer, output, numconf, add_ref):
    polymer = Chem.MolFromMolFile('porphyrine_C0.mol')
    print(f'Without H = {polymer.GetNumAtoms()}')
    mol = Chem.AddHs(polymer, addCoords=True)
    print(f'With H = {mol.GetNumAtoms()}')
    refmol = Chem.AddHs(Chem.Mol(mol))
    print(f'refmol = {refmol.GetNumAtoms()}')
    print(Chem.MolToMolBlock(refmol))
    param = rdDistGeom.ETKDGv2()

    # Delete repeated polymers
    #param.pruneRmsThresh = prunermsthresh
    #cids = rdDistGeom.EmbedMultipleConfs(mol, numconf, param)
    cids = rdDistGeom.EmbedMultipleConfs(mol, numconf, useRandomCoord=True)

    mp = AllChem.MMFFGetMoleculeProperties(mol, mmffVariant='MMFF94s')
    AllChem.MMFFOptimizeMoleculeConfs(mol, numThreads=0, mmffVariant='MMFF94s')
    w = Chem.SDWriter(output)
    if add_ref:
        refmol.SetProp('CID', '-1')
        refmol.SetProp('Energy', '')
        w.write(refmol)
    res = []
    
    for cid in cids:
        ff = AllChem.MMFFGetMoleculeForceField(mol, mp, confId=cid)
        e = ff.CalcEnergy()
        res.append((cid, e))
    sorted_res = sorted(res, key=lambda x: x[1])
    rdMolAlign.AlignMolConformers(mol)
    for cid, e in sorted_res:
        mol.SetProp('CID', str(cid))
        mol.SetProp('Energy', str(e))
        w.write(mol, confId=cid)
    w.close()


In [38]:
confgen(polymer=porphyrin_C0, output=f'blocks_conformers_porphyrin.sdf', numconf=1, add_ref=False)
suppl = Chem.ForwardSDMolSupplier(f'blocks_conformers_porphyrin.sdf')

for mol in suppl:
    logger.info(f"Writing xyz file of porphyrin")
    mol_H = Chem.AddHs(mol, addCoords=True)
    refmol_xyz = Chem.AddHs(Chem.Mol(mol_H))
    mol_xyz = Chem.MolToXYZBlock(refmol_xyz)
    initial = open(f"initial_porphyrin.xyz", "w")
    initial.writelines(mol_xyz)
    initial.close()

    

OSError: Bad input file porphyrine_C0.mol

## Porphyrin with Linker

In [39]:
linker_C1 = stk.BuildingBlock(
    smiles='C(Br)Br',
    functional_groups=[stk.BromoFactory()]
)

linker_C2 = stk.BuildingBlock(
    smiles='C(CBr)Br',
    functional_groups=[stk.BromoFactory()]
)


In [53]:
bb2_C1 = stk.ConstructedMolecule(
    topology_graph=stk.polymer.Linear((bb1,linker_C1,), 'AB', 4),
)

bb2_C1 = stk.BuildingBlock(
    smiles=stk.Smiles().get_key(bb2_C1),
    functional_groups=[stk.BromoFactory()],
)

bb2_C2 = stk.ConstructedMolecule(
    topology_graph=stk.polymer.Linear((bb1,linker_C2,), 'AB', 4),
)

bb2_C2 = stk.BuildingBlock(
    smiles=stk.Smiles().get_key(bb2_C2),
    functional_groups=[stk.BromoFactory()],
)



/Applications/anaconda3/lib/python3.8/site-packages/stk/utilities/utilities.py:455: RuntimeWarning: invalid value encountered in true_divide
  return np.divide(vector, np.linalg.norm(vector))


In [54]:
porphyrin_C1 = stk.ConstructedMolecule(
    topology_graph=stk.polymer.Linear(
        building_blocks=(bb2_C1, bb3),
        repeating_unit='BAB',
        num_repeating_units=1,
        orientations=(0, 0,1),
    ),
)

rdkit_porphyrin_C1 = porphyrin_C1.to_rdkit_mol()
rdkit.SanitizeMol(rdkit_porphyrin_C1)
rdkit.MMFFOptimizeMolecule(rdkit_porphyrin_C1)

# stk molecules are immutable. with_position_matrix returns a
# a clone, holding the new position matrix.
porphyrin_C1 = porphyrin_C1.with_position_matrix(
    position_matrix=rdkit_porphyrin_C1.GetConformer().GetPositions(),
)

show_stk_mol(porphyrin_C1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [55]:
porphyrin_C2 = stk.ConstructedMolecule(
    topology_graph=stk.polymer.Linear(
        building_blocks=(bb2_C2, bb3),
        repeating_unit='BAB',
        num_repeating_units=1,
        orientations=(0, 0,1),
    ),
)

rdkit_porphyrin_C2 = porphyrin_C2.to_rdkit_mol()
rdkit.SanitizeMol(rdkit_porphyrin_C2)
rdkit.MMFFOptimizeMolecule(rdkit_porphyrin_C2)

# stk molecules are immutable. with_position_matrix returns a
# a clone, holding the new position matrix.
porphyrin_C2 = porphyrin_C2.with_position_matrix(
    position_matrix=rdkit_porphyrin_C2.GetConformer().GetPositions(),
)

show_stk_mol(porphyrin_C2)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol